### 0.导入包并设置GPU

In [1]:
import os
import numpy as np

#设置LOG信息显示等级
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIABLE_DEVICES']='O'

import tensorflow as tf 
print("TensorFlow version:{}".format(tf.__version__))
print("Keras version:{}".format(tf.keras.__version__))

phy_gpus = tf.config.experimental.list_physical_devices('GPU')
print(type(phy_gpus),type(phy_gpus[0]))
print("Num of GPUs:",len(phy_gpus))
for gpu in phy_gpus:
    #设置GPU按需申请显存
    tf.config.experimental.set_memory_growth(gpu,True)

TensorFlow version:2.3.0
Keras version:2.4.0
<class 'list'> <class 'tensorflow.python.eager.context.PhysicalDevice'>
Num of GPUs: 1


### 1.创建Tensor

**Tensor的本质其实是一套计算的流程**。

* 在TensorFlow2.0中之所以定义好这个流程之后,计算结果也出来了，是因为EagerExecution这种模式。
* 在EagerExecution之前，Tensor都是只定义，所以在tf1.x中都是先定义出计算图，在需要计算的时候再通过session来执行计算。
* 在EagerExecution之后，Tensor定义好后，结果也计算好了。

In [2]:
#检查是否是Eager execution
print("Eager execution is:{}".format(tf.executing_eagerly()))

Eager execution is:True


In [3]:
#a和b都是tf.Tensor，并且在执行完b=a*a后计算结果就已经产生了。
a = tf.constant([[1,2],[3,4]])
b = a*a
a,b,type(a),type(b)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[ 1,  4],
        [ 9, 16]])>,
 tensorflow.python.framework.ops.EagerTensor,
 tensorflow.python.framework.ops.EagerTensor)

###  2.Tensor vs ndarray

* Numpy ndarray可以作为整个计算流程的输入，也可以作为整个计算流程的输出。
* 但是注意，**它只能作为输入和输出，进入TensorFlow计算流程之后都要用Tensor。**
* 除非一些之后不在计算流程中使用的Tensor可以通过tf.Tensor.numpy()方法将Tensor以ndarray导出。

In [4]:
c = np.array([[1,2],[3,4]])
d = tf.constant(c)
c,d,type(c),type(d)

(array([[1, 2],
        [3, 4]]),
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 numpy.ndarray,
 tensorflow.python.framework.ops.EagerTensor)

### 3.Tensor->ndarray

In [5]:
e = d.numpy()
e,type(e)

(array([[1, 2],
        [3, 4]]),
 numpy.ndarray)

### 4.cpu or gpu

In [6]:
print(d.device)
print(tf.config.list_physical_devices('CPU'))
print(tf.config.list_physical_devices('GPU'))

/job:localhost/replica:0/task:0/device:CPU:0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### 5.tf.Variable

tf.Variable是在Tensor基础上的进一步封装，Variable可以当成Tensor处理，但是Tensor不一定能当作Variable。

In [7]:
f = tf.Variable([[1,2],[3,4]])
f,type(f)

(<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
 array([[1, 2],
        [3, 4]])>,
 tensorflow.python.ops.resource_variable_ops.ResourceVariable)

### 6.修改Tensor的值

 tf.constant是默认无需从外部手动赋值的，而这时候如果又想手动修改Tensor的值，于是经过一层对Tensor的封装，可以手动修改的tf.Variable就诞生了。
#### tf.constant不可修改

In [8]:
g = tf.constant(100)
print(g)
g.assign(20)

tf.Tensor(100, shape=(), dtype=int32)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

#### tf.Variable可以通过assign方法修改

In [9]:
h = tf.Variable(100)
print(h)
h.assign(50)
print(h)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=100>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=50>


### 7.总结

Numpy ndarray VS tf.Tensor VS tf.Variable：

* 从前往后，逐层封装，增加功能，ndarray不可以放在GPU或者TPU上，后两者则可以。
* tf.Tensor不可手动更改，tf.Variable可以手动更改。
* tf.Variable是面向用户使用，tf.Tensor面向Tensorflow计算流程内部。